In [42]:
import pandas as pd
import boto3
import logging
from botocore.exceptions import ClientError
import os
from pprint import pprint


s3_client = boto3.client('s3',region_name='us-east-2')
textract = boto3.client('textract', region_name='us-east-2')

   

In [4]:
def create_bucket(bucket_name, region=None):
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [13]:
create_bucket('textract-ocr-2021', "us-east-2")

True

In [5]:
def upload_file(file_name, bucket, object_name=None):

    if object_name is None:
        object_name = os.path.basename(file_name)
    
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
        
    except ClientError as e:
        logging.error(e)
        return False
    
    return True

In [6]:
def extract_table(file_name, bucket):
    
    response = textract.start_document_analysis(
        DocumentLocation={
            'S3Object': {
                'Bucket': bucket,
                'Name': file_name
            }
        },
        FeatureTypes=["TABLES"]
    )
    
    
    blocks=response['Blocks']
    blocks_map = {}
    table_blocks = []
    
    for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)
            
    if len(table_blocks) <= 0:
        return "NO TABLE FOUND"
    
    return table_blocks

In [7]:
response = s3_client.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  textract-console-us-east-2-ae857a57-435c-46de-ba4c-fa07598fd9d5


In [8]:
upload_file('C:\\Users\\abhin\\OneDrive\\Documents\\Optab\\Data\\demo.pdf', 'textract-console-us-east-2-ae857a57-435c-46de-ba4c-fa07598fd9d5')

True

In [14]:
upload_file('C:\\Users\\abhin\\OneDrive\\Documents\\Optab\\Data\\demo.pdf', 'textract-ocr-2021')

True

In [71]:

def extract_table_blocks(bucket, file_name):
    response = textract.analyze_document(
        Document={
            'S3Object': {
                'Bucket': bucket,
                'Name': file_name
            }
        },
        FeatureTypes=["TABLES"]
    )
    
    return response
    


In [68]:
def get_rows_columns_map(table_result, blocks_map):
    rows = {}
    scores = []
    for relationship in table_result['Relationships']:
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = blocks_map[child_id]
                if cell['BlockType'] == 'CELL':
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    if row_index not in rows:
                        # create new row
                        rows[row_index] = {}
                    
                    # get confidence score
                    scores.append(str(cell['Confidence']))
                        
                    # get the text value
                    rows[row_index][col_index] = get_text(cell, blocks_map)
                    
    return rows, scores

In [55]:
def get_text(result, blocks_map):
    text = ''
    if 'Relationships' in result:
        for relationship in result['Relationships']:
            if relationship['Type'] == 'CHILD':
                for child_id in relationship['Ids']:
                    word = blocks_map[child_id]
                    if word['BlockType'] == 'WORD':
                        if "," in word['Text'] and word['Text'].replace(",", "").isnumeric():
                            text += '"' + word['Text'] + '"' + ' '
                        else:
                            text += word['Text'] + ' '
                    if word['BlockType'] == 'SELECTION_ELEMENT':
                        if word['SelectionStatus'] =='SELECTED':
                            text +=  'X '
    return text

In [72]:

response = extract_table_blocks('textract-ocr-2021', 'demo.pdf')
blocks=response['Blocks']
blocks_map = {}
table_blocks = []
    
for block in blocks:
        blocks_map[block['Id']] = block
        if block['BlockType'] == "TABLE":
            table_blocks.append(block)
            
for index, table in enumerate(table_blocks):
        rows, scores = get_rows_columns_map(table, blocks_map)
        

print(rows) 


{1: {1: 'Item ', 2: 'Quantity ', 3: 'Unit ', 4: 'Description ', 5: 'Transaction Date ', 6: 'Unit Excluding VAT ', 7: 'TOTAL EXCLUDING VAT ', 8: 'VAT ', 9: 'Total Including VAT ', 10: 'VAT CODE '}, 2: {1: '40043 ', 2: '4 ', 3: 'Hours ', 4: 'Mechanical Engineer ', 5: '23 Feb 24 ', 6: '€140.00 ', 7: '€560.00 ', 8: '€128.80 ', 9: '€688.80 ', 10: 'IRE '}, 3: {1: '40043 ', 2: '0 ', 3: 'Hours ', 4: 'Mechanical Engineer ', 5: '24 Feb 24 ', 6: '€160.00 ', 7: '€0.00 ', 8: '€0.00 ', 9: '€0.00 ', 10: 'IRE '}, 4: {1: '40043 ', 2: '0 ', 3: 'Hours ', 4: 'Mechanical Engineer ', 5: '25 Feb 24 ', 6: '€106.00 ', 7: '€0.00 ', 8: '€0.00 ', 9: '€0.00 ', 10: 'IRE '}, 5: {1: '40043 ', 2: '5 ', 3: 'Hours ', 4: 'Mechanical Engineer ', 5: '26 Feb 24 ', 6: '€139.00 ', 7: '€695.00 ', 8: '€159.85 ', 9: '€854.85 ', 10: 'IRE '}, 6: {1: '40043 ', 2: '5 ', 3: 'Hours ', 4: 'Mechanical Engineer ', 5: '27 Feb 24 ', 6: '€159.00 ', 7: '€795.00 ', 8: '€182.85 ', 9: '€977.85 ', 10: 'IRE '}, 7: {1: '40043 ', 2: '2 ', 3: 'Hours

In [73]:
table_id = 'Table_' + str("Table_Demo")
csv = 'Table: {0}\n\n'.format(table_id)

for row_index, cols in rows.items():
        for col_index, text in cols.items():
            col_indices = len(cols.items())
            csv += '{}'.format(text) + ","
        csv += '\n'
        

print(csv)


Table: Table_Table_Demo

Item ,Quantity ,Unit ,Description ,Transaction Date ,Unit Excluding VAT ,TOTAL EXCLUDING VAT ,VAT ,Total Including VAT ,VAT CODE ,
40043 ,4 ,Hours ,Mechanical Engineer ,23 Feb 24 ,€140.00 ,€560.00 ,€128.80 ,€688.80 ,IRE ,
40043 ,0 ,Hours ,Mechanical Engineer ,24 Feb 24 ,€160.00 ,€0.00 ,€0.00 ,€0.00 ,IRE ,
40043 ,0 ,Hours ,Mechanical Engineer ,25 Feb 24 ,€106.00 ,€0.00 ,€0.00 ,€0.00 ,IRE ,
40043 ,5 ,Hours ,Mechanical Engineer ,26 Feb 24 ,€139.00 ,€695.00 ,€159.85 ,€854.85 ,IRE ,
40043 ,5 ,Hours ,Mechanical Engineer ,27 Feb 24 ,€159.00 ,€795.00 ,€182.85 ,€977.85 ,IRE ,
40043 ,2 ,Hours ,Mechanical Engineer ,28 Feb 24 ,€155.00 ,€310.00 ,€71.30 ,€381.30 ,IRE ,
40043 ,5 ,Hours ,Mechanical Engineer ,29 Feb 24 ,€162.00 ,€810.00 ,€186.30 ,€996.30 ,IRE ,
40043 ,4 ,Hours ,Mechanical Engineer ,1 Mar 24 ,€162.00 ,€648.00 ,€149.04 ,€797.04 ,IRE ,
40043 ,0 ,Hours ,Mechanical Engineer ,2 24 ,€154.00 ,€0.00 ,€0.00 ,€0.00 ,IRE ,
40043 ,0 ,Hours ,Mechanical Engineer ,3 Mar 24 ,€1

In [78]:
import sys

output_file = os.path.join("C:\\Users\\abhin\\OneDrive\\Documents\\Optab", 'demo_invoice.csv')


# Write the file to the custom directory
with open(output_file, "wt") as fout:
    fout.write(csv)

'c:\\Users\\abhin\\AppData\\Local\\Programs\\Microsoft VS Code'